In [12]:
import pandas as pd

# List of CSV files to combine
csv_files = [
    'EIA930_SUBREGION_2018_Jul_Dec.csv',
    'EIA930_SUBREGION_2019_Jan_Jun.csv',
    'EIA930_SUBREGION_2019_Jul_Dec.csv',
    'EIA930_SUBREGION_2020_Jan_Jun.csv'
]

# Read and concatenate all CSVs
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Save to a new CSV
combined_df.to_csv('merged_region.csv', index=False)

print("CSV files combined and saved")


CSV files combined and saved


In [15]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_region.csv')

# Fill missing values in 'Demand (MW)' using forward fill
df['Demand (MW)'] = df['Demand (MW)'].fillna(method='ffill')

# Save the updated dataset
df.to_csv('merged_region.csv', index=False)

print("Missing values in 'Demand (MW)' filled using forward fill and file updated.")


C:\Users\leads trading\AppData\Local\Temp\ipykernel_13704\1745256236.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Demand (MW)'] = df['Demand (MW)'].fillna(method='ffill')


Missing values in 'Demand (MW)' filled using forward fill and file updated.


In [16]:
import pandas as pd


df = pd.read_csv('merged_region.csv')


print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1355952 entries, 0 to 1355951
Data columns (total 7 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   Balancing Authority        1355952 non-null  object
 1   Data Date                  1355952 non-null  object
 2   Hour Number                1355952 non-null  int64 
 3   Sub-Region                 1355952 non-null  object
 4   Demand (MW)                1355952 non-null  object
 5   Local Time at End of Hour  1355952 non-null  object
 6   UTC Time at End of Hour    1355952 non-null  object
dtypes: int64(1), object(6)
memory usage: 72.4+ MB
None
Balancing Authority          0
Data Date                    0
Hour Number                  0
Sub-Region                   0
Demand (MW)                  0
Local Time at End of Hour    0
UTC Time at End of Hour      0
dtype: int64


In [20]:
import pandas as pd

# Load the dataset
merged_region_df = pd.read_csv('merged_region.csv')

# Get the unique sub-regions
unique_subregions = merged_region_df['Sub-Region'].unique()

# Print the unique sub-regions
print('Unique Sub-Regions:')
for subregion in unique_subregions:
    print(subregion)



Unique Sub-Regions:
PGAE
SCE
SDGE
VEA
COAS
EAST
FWES
NCEN
NRTH
SCEN
SOUT
WEST
4001
4002
4003
4004
4005
4006
4007
4008
0001
0004
0006
0027
0035
8910
ZONA
ZONB
ZONC
ZOND
ZONE
ZONF
ZONG
ZONH
ZONI
ZONJ
ZONK
AE
AEP
AP
ATSI
BC
CE
DAY
DEOK
DOM
DPL
DUQ
EKPC
JC
ME
PE
PEP
PL
PN
PS
RECO
Frep
Jica
KAFB
KCEC
LAC
NTUA
PNM
TSGT
CSWS
EDE
GRDA
INDN
KACY
KCPL
LES
MPS
NPPD
OKGE
OPPD
SECI
SPRM
SPS
WAUE
WFEC
WR
1
4
6
27
35


In [2]:
import pandas as pd
import json
import numpy as np

# Step 1: Load and clean the demand data
merged_region = pd.read_csv('merged_region.csv')
merged_region['Demand (MW)'] = merged_region['Demand (MW)'].str.replace(',', '').astype(float)
merged_region['Local Time at End of Hour'] = pd.to_datetime(
    merged_region['Local Time at End of Hour'], 
    format='mixed', 
    errors='coerce'
)
merged_region = merged_region.dropna(subset=['Local Time at End of Hour'])

# Step 2: Load all weather data
weather_files = [
    'dallas.json', 'houston.json', 'la.json', 'nyc.json', 'philadelphia.json',
    'phoenix.json', 'san_antonio.json', 'san_diego.json', 'san_jose.json', 'seattle.json'
]

weather_data = {}
for file in weather_files:
    city_name = file.split('.')[0]
    try:
        with open(file, 'r') as f:
            weather_data[city_name] = json.load(f)
    except FileNotFoundError:
        print(f"Warning: File {file} not found. Skipping.")
        continue

# Step 3: Create weather DataFrames
weather_dfs = {}
for city, data in weather_data.items():
    df = pd.DataFrame(data)
    df['datetime'] = pd.to_datetime(df['time'], unit='s')
    df['city'] = city
    weather_dfs[city] = df

# Step 4: Define city mappings with multiple options for Seattle
city_mappings = {
    # California
    'la': [('CISO', 'SCE')],
    'san_diego': [('CISO', 'SDGE')],
    'san_jose': [('CISO', 'PGAE')],
    
    # Texas
    'dallas': [('ERCO', 'NRTH')],
    'houston': [('ERCO', 'SCEN')],
    'san_antonio': [('ERCO', 'SOUT')],
    
    # Northeast
    'nyc': [('NYIS', 'ZONA'), ('NYIS', '0001')],
    'philadelphia': [('PJM', 'PE')],
    
    # Southwest
    'phoenix': [('PNM', 'PNM'), ('AZPS', 'AZPS')],
    
    # Northwest - multiple options for Seattle
    'seattle': [
        ('MISO', 'WEST'),
        ('NWPP', 'WAUW'),
        ('BPAT', 'WAUE'),
        ('PJM', 'PS')
    ]
}

# Step 5: Match cities with demand data
matched_data = []

for city, ba_subregion_options in city_mappings.items():
    if city not in weather_dfs:
        print(f"Weather data not found for {city}")
        continue
        
    weather_df = weather_dfs[city]
    matched_for_city = False
    
    for ba, sub_region in ba_subregion_options:
        matching_demand = merged_region[
            (merged_region['Balancing Authority'] == ba) & 
            (merged_region['Sub-Region'] == sub_region)
        ]
        
        if matching_demand.empty:
            continue
            
        # Found matching demand data
        matched_for_city = True
        print(f"Matched {city} with BA={ba}, Sub-Region={sub_region}")
        
        for _, demand_row in matching_demand.iterrows():
            local_time = demand_row['Local Time at End of Hour']
            time_diff = (weather_df['datetime'] - local_time).abs()
            closest_idx = time_diff.idxmin()
            closest_time_diff = time_diff[closest_idx].total_seconds() / 3600
            
            if closest_time_diff <= 1:
                weather_row = weather_df.loc[closest_idx]
                
                merged_row = {
                    'Balancing Authority': ba,
                    'Data Date': demand_row['Data Date'],
                    'Hour Number': demand_row['Hour Number'],
                    'Sub-Region': sub_region,
                    'Demand (MW)': demand_row['Demand (MW)'],
                    'Local Time': local_time,
                    'City': city,
                    'Temperature (F)': weather_row['temperature'],
                    'Humidity': weather_row.get('humidity', np.nan),
                    'Wind Speed (mph)': weather_row.get('windSpeed', np.nan),
                    'Weather Summary': weather_row.get('summary', ''),
                    'Pressure': weather_row.get('pressure', np.nan),
                    'UV Index': weather_row.get('uvIndex', np.nan),
                    'Visibility': weather_row.get('visibility', np.nan),
                    'Cloud Cover': weather_row.get('cloudCover', np.nan)
                }
                
                matched_data.append(merged_row)
        
        break  # Stop after first successful match
    
    if not matched_for_city:
        print(f"Could not match {city} with any BA/Sub-Region combination")

# Step 6: Create final DataFrame and add features
final_df = pd.DataFrame(matched_data)

if not final_df.empty:
    # Add time-based features
    final_df['Hour'] = final_df['Local Time'].dt.hour
    final_df['Day'] = final_df['Local Time'].dt.day
    final_df['Month'] = final_df['Local Time'].dt.month
    final_df['Weekday'] = final_df['Local Time'].dt.dayofweek
    final_df['Weekend'] = final_df['Weekday'].apply(lambda x: 1 if x >= 5 else 0)
    final_df['Season'] = final_df['Month'].apply(
        lambda x: 1 if x in [12,1,2] else 2 if x in [3,4,5] else 3 if x in [6,7,8] else 4
    )
    
    # Save to CSV
    final_df.to_csv('merged_weather_demand_final.csv', index=False)
    
    # Print summary
    print("\nSuccessfully created dataset with", len(final_df), "rows")
    print("Cities included:", final_df['City'].unique())
    print("\nRows per city:")
    print(final_df['City'].value_counts())
else:
    print("No matching data was found. Please check your data files and mappings.")

Matched la with BA=CISO, Sub-Region=SCE
Matched san_diego with BA=CISO, Sub-Region=SDGE
Matched san_jose with BA=CISO, Sub-Region=PGAE
Matched dallas with BA=ERCO, Sub-Region=NRTH
Matched houston with BA=ERCO, Sub-Region=SCEN
Matched san_antonio with BA=ERCO, Sub-Region=SOUT
Matched nyc with BA=NYIS, Sub-Region=ZONA
Matched philadelphia with BA=PJM, Sub-Region=PE
Matched phoenix with BA=PNM, Sub-Region=PNM
Matched seattle with BA=PJM, Sub-Region=PS

Successfully created dataset with 165202 rows
Cities included: ['la' 'san_diego' 'san_jose' 'dallas' 'houston' 'san_antonio' 'nyc'
 'philadelphia' 'phoenix' 'seattle']

Rows per city:
City
la              16531
san_diego       16531
san_jose        16531
phoenix         16531
seattle         16531
nyc             16510
philadelphia    16510
dallas          16509
houston         16509
san_antonio     16509
Name: count, dtype: int64


In [3]:
import pandas as pd

# Load the merged dataset
df = pd.read_csv('merged_weather_demand_final.csv')

# 1. Check for missing values in each column
print("="*50)
print("Missing Values Per Column:")
print("="*50)
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])  # Only show columns with missing values

# 2. Check percentage of missing values
print("\n" + "="*50)
print("Missing Values Percentage:")
print("="*50)
missing_percentage = (df.isnull().mean() * 100).round(2)
print(missing_percentage[missing_percentage > 0])  # Only show columns with missing values

# 3. Check for complete cases (rows with no missing values)
print("\n" + "="*50)
print("Complete Cases Analysis:")
print("="*50)
complete_cases = df.dropna()
print(f"Total rows: {len(df)}")
print(f"Complete rows (no missing values): {len(complete_cases)}")
print(f"Percentage complete: {len(complete_cases)/len(df)*100:.2f}%")

# 4. Check for missing values by city
print("\n" + "="*50)
print("Missing Values By City:")
print("="*50)
if 'City' in df.columns:
    for city in df['City'].unique():
        city_df = df[df['City'] == city]
        city_missing = city_df.isnull().sum().sum()
        print(f"{city}: {city_missing} missing values")

# 5. Detailed missing data pattern
print("\n" + "="*50)
print("Detailed Missing Data Pattern:")
print("="*50)
print("Columns with missing values and sample affected rows:")
for col in df.columns:
    if df[col].isnull().any():
        print(f"\nColumn: {col}")
        print(f"Missing count: {df[col].isnull().sum()}")
        print("Sample rows with missing values:")
        print(df[df[col].isnull()][['City', 'Local Time', col]].head(3))

# Optional: Visualize missing data (requires matplotlib)
try:
    import matplotlib.pyplot as plt
    import missingno as msno
    
    print("\nGenerating missing data visualization...")
    msno.matrix(df)
    plt.title('Missing Data Pattern')
    plt.show()
    
    msno.bar(df)
    plt.title('Missing Values Per Column')
    plt.show()
except ImportError:
    print("\nFor visualizations, install missingno: pip install missingno")

Missing Values Per Column:
Temperature (F)      10
Humidity             10
Wind Speed (mph)     53
Weather Summary     277
Pressure             50
UV Index            175
Visibility          172
Cloud Cover         240
dtype: int64

Missing Values Percentage:
Temperature (F)     0.01
Humidity            0.01
Wind Speed (mph)    0.03
Weather Summary     0.17
Pressure            0.03
UV Index            0.11
Visibility          0.10
Cloud Cover         0.15
dtype: float64

Complete Cases Analysis:
Total rows: 165202
Complete rows (no missing values): 164869
Percentage complete: 99.80%

Missing Values By City:
la: 98 missing values
san_diego: 99 missing values
san_jose: 99 missing values
dallas: 99 missing values
houston: 100 missing values
san_antonio: 100 missing values
nyc: 97 missing values
philadelphia: 96 missing values
phoenix: 98 missing values
seattle: 101 missing values

Detailed Missing Data Pattern:
Columns with missing values and sample affected rows:

Column: Temperature (F)

In [3]:
import pandas as pd

# Load the dataset and parse 'Local Time' as datetime
df = pd.read_csv('merged_weather_demand_final.csv', parse_dates=['Local Time'])

# Set 'Local Time' as the datetime index for time-aware operations
df.set_index('Local Time', inplace=True)

# List of numeric columns to interpolate
num_cols = ['Temperature (F)', 'Humidity', 'Wind Speed (mph)', 
            'Pressure', 'UV Index', 'Visibility', 'Cloud Cover']

# Time-based interpolation for numeric values
df[num_cols] = df[num_cols].interpolate(method='time')

# Forward fill for the categorical column
df['Weather Summary'] = df['Weather Summary'].fillna(method='ffill')

# Backfill any remaining missing values (just in case)
df = df.fillna(method='bfill')

# Reset index to save 'Local Time' back as a column
df.reset_index(inplace=True)

# Save the cleaned dataset (overwrite the file)
df.to_csv('merged_weather_demand_final.csv', index=False)

print("✅ Missing values handled using time-aware methods and file saved.")


C:\Users\leads trading\AppData\Local\Temp\ipykernel_14780\2189600972.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Weather Summary'] = df['Weather Summary'].fillna(method='ffill')
C:\Users\leads trading\AppData\Local\Temp\ipykernel_14780\2189600972.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')


✅ Missing values handled using time-aware methods and file saved.


In [4]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('merged_weather_demand_final.csv')

# Check for missing values
missing_values = df.isnull().sum()

# Print columns with missing values (if any)
print("Missing values in each column:")
print(missing_values[missing_values > 0])


Missing values in each column:
Series([], dtype: int64)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
print("Loading dataset...")
df = pd.read_csv('merged_weather_demand_final.csv')

# Display basic info
print(f"\nDataset shape: {df.shape}")
print(f"First few rows:")
print(df.head(2))


# Convert to datetime using 'Data Date' and 'Hour Number'
print("\nConverting date and time columns...")
df['DateTime'] = pd.to_datetime(df['Data Date'] + ' ' + df['Hour Number'].astype(str) + ':00', errors='coerce')

# Check if DateTime conversion was successful
print(f"DateTime conversion successful: {not df['DateTime'].isna().all()}")
print(f"Sample DateTime values: {df['DateTime'].head(3).tolist()}")

# 1. CHECK TIME INTERVAL CONSISTENCY
print("\n==== CHECKING TIME INTERVAL CONSISTENCY ====")

def check_time_intervals(city_data):
    # Sort by datetime
    city_data = city_data.sort_values('DateTime')
    # Calculate time differences between consecutive rows
    time_diffs = city_data['DateTime'].diff().dropna()
    
    if len(time_diffs) == 0:
        return {
            'consistent': None,
            'modal_interval': None,
            'irregular_count': 0,
            'irregular_intervals': None
        }
    
    # Most common time difference (should be 1 hour)
    modal_diff = time_diffs.mode()[0]
    # Check if all differences are the same
    consistent = (time_diffs == modal_diff).all()
    # Find any irregular intervals
    irregular = time_diffs[time_diffs != modal_diff]
    return {
        'consistent': consistent,
        'modal_interval': modal_diff,
        'irregular_count': len(irregular),
        'irregular_intervals': irregular if len(irregular) > 0 else None
    }

# Apply check to each city
time_consistency = {}
for city in df['City'].unique():
    city_data = df[df['City'] == city]
    time_consistency[city] = check_time_intervals(city_data)
    
print("Time interval consistency by city:")
for city, results in time_consistency.items():
    print(f"\n{city}:")
    print(f"  Consistent intervals: {results['consistent']}")
    print(f"  Expected interval: {results['modal_interval']}")
    if results['irregular_count'] > 0:
        print(f"  Found {results['irregular_count']} irregular intervals")
        # Show sample of irregular intervals
        sample_irregulars = results['irregular_intervals'].head(5)
        print(f"  Sample irregular intervals: {sample_irregulars.tolist()}")

# 2. CHECK FOR DUPLICATES
print("\n==== CHECKING FOR DUPLICATES ====")

duplicates = df.duplicated(subset=['City', 'DateTime'], keep=False)
duplicate_records = df[duplicates].sort_values(['City', 'DateTime'])

if len(duplicate_records) > 0:
    print(f"Found {len(duplicate_records)} duplicate records:")
    print(duplicate_records.head(10))  # Show first 10 duplicates
    
    # Count duplicates by city
    dup_by_city = duplicate_records.groupby('City').size()
    print("\nDuplicates by city:")
    print(dup_by_city)
else:
    print("No duplicate records found.")

# 3. DETECT ANOMALIES
print("\n==== DETECTING ANOMALIES ====")

# Approach 1: Statistical method (Z-score) for demand anomalies by city
def detect_demand_anomalies(df, threshold=3.0):
    anomalies = pd.DataFrame()
    anomaly_counts = {}
    
    for city in df['City'].unique():
        city_data = df[df['City'] == city].copy()
        
        # Calculate z-scores for demand
        city_data['demand_zscore'] = np.abs((city_data['Demand (MW)'] - 
                                          city_data['Demand (MW)'].mean()) / 
                                         city_data['Demand (MW)'].std())
        
        # Flag anomalies
        city_anomalies = city_data[city_data['demand_zscore'] > threshold]
        anomalies = pd.concat([anomalies, city_anomalies])
        anomaly_counts[city] = len(city_anomalies)
    
    return anomalies, anomaly_counts

# Approach 2: IQR method for weather variables
def detect_weather_anomalies(df):
    anomalies = pd.DataFrame()
    anomaly_counts = {}
    weather_vars = ['Temperature (F)', 'Humidity', 'Wind Speed (mph)', 'Pressure']
    
    for city in df['City'].unique():
        city_data = df[df['City'] == city].copy()
        city_anomalies = pd.DataFrame()
        city_count = 0
        
        for var in weather_vars:
            if var in city_data.columns:
                Q1 = city_data[var].quantile(0.25)
                Q3 = city_data[var].quantile(0.75)
                IQR = Q3 - Q1
                
                # Define bounds
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                # Flag outliers
                outliers = city_data[(city_data[var] < lower_bound) | 
                                   (city_data[var] > upper_bound)].copy()
                
                if len(outliers) > 0:
                    outliers['anomaly_variable'] = var
                    outliers['lower_bound'] = lower_bound
                    outliers['upper_bound'] = upper_bound
                    city_anomalies = pd.concat([city_anomalies, outliers])
                    city_count += len(outliers)
        
        if city_count > 0:
            anomalies = pd.concat([anomalies, city_anomalies])
            anomaly_counts[city] = city_count
    
    return anomalies, anomaly_counts

# Approach 3: Check for physically impossible values
def check_physical_constraints(df):
    impossible_values = pd.DataFrame()
    constraint_violations = {}
    
    # Define logical constraints
    constraints = {
        'Temperature (F)': (lambda x: (x < -50) | (x > 130), "Temperature outside -50°F to 130°F"),
        'Humidity': (lambda x: (x < 0) | (x > 1), "Humidity outside 0-1 range"),
        'Wind Speed (mph)': (lambda x: (x < 0) | (x > 200), "Wind speed outside 0-200 mph"),
        'Pressure': (lambda x: (x < 900) | (x > 1100), "Pressure outside 900-1100 range"),
        'Demand (MW)': (lambda x: x < 0, "Negative demand")
    }
    
    for var, (constraint, msg) in constraints.items():
        if var in df.columns:
            invalid = df[constraint(df[var])].copy()
            if len(invalid) > 0:
                invalid['constraint_violation'] = msg
                impossible_values = pd.concat([impossible_values, invalid])
                constraint_violations[var] = len(invalid)
    
    return impossible_values, constraint_violations

# Run the anomaly detection
print("Detecting demand anomalies (using Z-score method)...")
demand_anomalies, demand_anomaly_counts = detect_demand_anomalies(df)

print("Detecting weather anomalies (using IQR method)...")
weather_anomalies, weather_anomaly_counts = detect_weather_anomalies(df)

print("Checking for physically impossible values...")
impossible_values, constraint_violations = check_physical_constraints(df)

print(f"\nFound {len(demand_anomalies)} demand anomalies")
if len(demand_anomalies) > 0:
    print("Demand anomalies by city:")
    for city, count in demand_anomaly_counts.items():
        print(f"  {city}: {count} anomalies")
    # Show sample of demand anomalies
    print("\nSample demand anomalies:")
    sample_cols = ['City', 'DateTime', 'Demand (MW)', 'demand_zscore']
    print(demand_anomalies[sample_cols].head(5))

print(f"\nFound {len(weather_anomalies)} weather anomalies")
if len(weather_anomalies) > 0:
    print("Weather anomalies by city:")
    for city, count in weather_anomaly_counts.items():
        print(f"  {city}: {count} anomalies")
    # Show sample of weather anomalies
    print("\nSample weather anomalies:")
    sample_cols = ['City', 'DateTime', 'anomaly_variable', 'lower_bound', 'upper_bound']
    print(weather_anomalies[sample_cols].head(5))

print(f"\nFound {len(impossible_values)} physically impossible values")
if len(impossible_values) > 0:
    print("Constraint violations by variable:")
    for var, count in constraint_violations.items():
        print(f"  {var}: {count} violations")
    # Show sample of impossible values
    print("\nSample impossible values:")
    sample_cols = ['City', 'DateTime', 'constraint_violation']
    print(impossible_values[sample_cols].head(5))

# 4. VISUALIZATION (save to files)
print("\n==== GENERATING VISUALIZATIONS ====")

# Plot demand time series with anomalies highlighted for a sample city
def plot_demand_anomalies(df, city, anomalies, save_path):
    plt.figure(figsize=(15, 6))
    
    city_data = df[df['City'] == city]
    city_anomalies = anomalies[anomalies['City'] == city]
    
    plt.plot(city_data['DateTime'], city_data['Demand (MW)'], 'b-', alpha=0.7)
    if len(city_anomalies) > 0:
        plt.scatter(city_anomalies['DateTime'], city_anomalies['Demand (MW)'], 
                    color='red', s=50, label='Anomalies')
    
    plt.title(f'Electricity Demand for {city} with Anomalies Highlighted')
    plt.xlabel('Date')
    plt.ylabel('Demand (MW)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"Saved demand anomaly plot for {city} to {save_path}")

# Boxplots for weather variables by city
def plot_weather_boxplots(df, save_path):
    weather_vars = ['Temperature (F)', 'Humidity', 'Wind Speed (mph)', 'Pressure']
    for var in weather_vars:
        if var not in df.columns:
            continue
            
        plt.figure(figsize=(14, 8))
        sns.boxplot(x='City', y=var, data=df)
        plt.title(f'Distribution of {var} by City')
        plt.xticks(rotation=45)
        plt.tight_layout()
        var_name = var.replace(" ", "_").replace("(", "").replace(")", "")
        file_path = f"{save_path}_{var_name}.png"
        plt.savefig(file_path)
        plt.close()
        print(f"Saved boxplot for {var} to {file_path}")

# Create visualizations for a few sample cities
if len(df['City'].unique()) > 0:
    for i, city in enumerate(df['City'].unique()[:3]):  # First 3 cities
        plot_demand_anomalies(df, city, demand_anomalies, f"demand_anomalies_{city}.png")
    
    plot_weather_boxplots(df, "weather_boxplot")

print("\nAnalysis complete! Check the output files for visualizations.")

Loading dataset...

Dataset shape: (165202, 21)
First few rows:
            Local Time Balancing Authority   Data Date  Hour Number  \
0  2018-07-01 06:00:00                CISO  07/01/2018            6   
1  2018-07-01 07:00:00                CISO  07/01/2018            7   

  Sub-Region  Demand (MW) City  Temperature (F)  Humidity  Wind Speed (mph)  \
0        SCE       9472.0   la            65.45      0.79              4.23   
1        SCE       9353.0   la            65.45      0.79              4.23   

   ... Pressure  UV Index  Visibility  Cloud Cover  Hour  Day  Month  Weekday  \
0  ...   1014.5       0.0       9.798         0.25     6    1      7        6   
1  ...   1014.5       0.0       9.798         0.25     7    1      7        6   

   Weekend  Season  
0        1       3  
1        1       3  

[2 rows x 21 columns]

Converting date and time columns...
DateTime conversion successful: True
Sample DateTime values: [Timestamp('2018-07-01 06:00:00'), Timestamp('2018-07-01

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
print("Loading dataset...")
df = pd.read_csv('merged_weather_demand_final.csv')

# Display basic info
print(f"\nDataset shape: {df.shape}")
print(f"First few rows:")
print(df.head(2))

# From the sample data you provided, these should be the column names
# Convert to datetime using 'Data Date' and 'Hour Number'
print("\nConverting date and time columns...")
df['DateTime'] = pd.to_datetime(df['Data Date'] + ' ' + df['Hour Number'].astype(str) + ':00', errors='coerce')

# Check if DateTime conversion was successful
print(f"DateTime conversion successful: {not df['DateTime'].isna().all()}")
print(f"Sample DateTime values: {df['DateTime'].head(3).tolist()}")

# 1. CHECK TIME INTERVAL CONSISTENCY
print("\n==== CHECKING TIME INTERVAL CONSISTENCY ====")

def check_time_intervals(city_data):
    # Sort by datetime
    city_data = city_data.sort_values('DateTime')
    # Calculate time differences between consecutive rows
    time_diffs = city_data['DateTime'].diff().dropna()
    
    if len(time_diffs) == 0:
        return {
            'consistent': None,
            'modal_interval': None,
            'irregular_count': 0,
            'irregular_intervals': None
        }
    
    # Most common time difference (should be 1 hour)
    modal_diff = time_diffs.mode()[0]
    # Check if all differences are the same
    consistent = (time_diffs == modal_diff).all()
    # Find any irregular intervals
    irregular = time_diffs[time_diffs != modal_diff]
    return {
        'consistent': consistent,
        'modal_interval': modal_diff,
        'irregular_count': len(irregular),
        'irregular_intervals': irregular if len(irregular) > 0 else None
    }

# Apply check to each city
time_consistency = {}
for city in df['City'].unique():
    city_data = df[df['City'] == city]
    time_consistency[city] = check_time_intervals(city_data)
    
print("Time interval consistency by city:")
for city, results in time_consistency.items():
    print(f"\n{city}:")
    print(f"  Consistent intervals: {results['consistent']}")
    print(f"  Expected interval: {results['modal_interval']}")
    if results['irregular_count'] > 0:
        print(f"  Found {results['irregular_count']} irregular intervals")
        # Show sample of irregular intervals
        sample_irregulars = results['irregular_intervals'].head(5)
        print(f"  Sample irregular intervals: {sample_irregulars.tolist()}")

# 2. CHECK FOR DUPLICATES
print("\n==== CHECKING FOR DUPLICATES ====")

duplicates = df.duplicated(subset=['City', 'DateTime'], keep=False)
duplicate_records = df[duplicates].sort_values(['City', 'DateTime'])

if len(duplicate_records) > 0:
    print(f"Found {len(duplicate_records)} duplicate records:")
    print(duplicate_records.head(10))  # Show first 10 duplicates
    
    # Count duplicates by city
    dup_by_city = duplicate_records.groupby('City').size()
    print("\nDuplicates by city:")
    print(dup_by_city)
else:
    print("No duplicate records found.")

# 3. DETECT ANOMALIES
print("\n==== DETECTING ANOMALIES ====")

# Approach 1: Statistical method (Z-score) for demand anomalies by city
def detect_demand_anomalies(df, threshold=3.0):
    anomalies = pd.DataFrame()
    anomaly_counts = {}
    
    for city in df['City'].unique():
        city_data = df[df['City'] == city].copy()
        
        # Calculate z-scores for demand
        city_data['demand_zscore'] = np.abs((city_data['Demand (MW)'] - 
                                          city_data['Demand (MW)'].mean()) / 
                                         city_data['Demand (MW)'].std())
        
        # Flag anomalies
        city_anomalies = city_data[city_data['demand_zscore'] > threshold]
        anomalies = pd.concat([anomalies, city_anomalies])
        anomaly_counts[city] = len(city_anomalies)
    
    return anomalies, anomaly_counts

# Approach 2: IQR method for weather variables
def detect_weather_anomalies(df):
    anomalies = pd.DataFrame()
    anomaly_counts = {}
    weather_vars = ['Temperature (F)', 'Humidity', 'Wind Speed (mph)', 'Pressure']
    
    for city in df['City'].unique():
        city_data = df[df['City'] == city].copy()
        city_anomalies = pd.DataFrame()
        city_count = 0
        
        for var in weather_vars:
            if var in city_data.columns:
                Q1 = city_data[var].quantile(0.25)
                Q3 = city_data[var].quantile(0.75)
                IQR = Q3 - Q1
                
                # Define bounds
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                # Flag outliers
                outliers = city_data[(city_data[var] < lower_bound) | 
                                   (city_data[var] > upper_bound)].copy()
                
                if len(outliers) > 0:
                    outliers['anomaly_variable'] = var
                    outliers['lower_bound'] = lower_bound
                    outliers['upper_bound'] = upper_bound
                    city_anomalies = pd.concat([city_anomalies, outliers])
                    city_count += len(outliers)
        
        if city_count > 0:
            anomalies = pd.concat([anomalies, city_anomalies])
            anomaly_counts[city] = city_count
    
    return anomalies, anomaly_counts

# Approach 3: Check for physically impossible values
def check_physical_constraints(df):
    impossible_values = pd.DataFrame()
    constraint_violations = {}
    
    # Define logical constraints
    constraints = {
        'Temperature (F)': (lambda x: (x < -50) | (x > 130), "Temperature outside -50°F to 130°F"),
        'Humidity': (lambda x: (x < 0) | (x > 1), "Humidity outside 0-1 range"),
        'Wind Speed (mph)': (lambda x: (x < 0) | (x > 200), "Wind speed outside 0-200 mph"),
        'Pressure': (lambda x: (x < 900) | (x > 1100), "Pressure outside 900-1100 range"),
        'Demand (MW)': (lambda x: x < 0, "Negative demand")
    }
    
    for var, (constraint, msg) in constraints.items():
        if var in df.columns:
            invalid = df[constraint(df[var])].copy()
            if len(invalid) > 0:
                invalid['constraint_violation'] = msg
                impossible_values = pd.concat([impossible_values, invalid])
                constraint_violations[var] = len(invalid)
    
    return impossible_values, constraint_violations

# Run the anomaly detection
print("Detecting demand anomalies (using Z-score method)...")
demand_anomalies, demand_anomaly_counts = detect_demand_anomalies(df)

print("Detecting weather anomalies (using IQR method)...")
weather_anomalies, weather_anomaly_counts = detect_weather_anomalies(df)

print("Checking for physically impossible values...")
impossible_values, constraint_violations = check_physical_constraints(df)

print(f"\nFound {len(demand_anomalies)} demand anomalies")
if len(demand_anomalies) > 0:
    print("Demand anomalies by city:")
    for city, count in demand_anomaly_counts.items():
        print(f"  {city}: {count} anomalies")
    # Show sample of demand anomalies
    print("\nSample demand anomalies:")
    sample_cols = ['City', 'DateTime', 'Demand (MW)', 'demand_zscore']
    print(demand_anomalies[sample_cols].head(5))

print(f"\nFound {len(weather_anomalies)} weather anomalies")
if len(weather_anomalies) > 0:
    print("Weather anomalies by city:")
    for city, count in weather_anomaly_counts.items():
        print(f"  {city}: {count} anomalies")
    # Show sample of weather anomalies
    print("\nSample weather anomalies:")
    sample_cols = ['City', 'DateTime', 'anomaly_variable', 'lower_bound', 'upper_bound']
    print(weather_anomalies[sample_cols].head(5))

print(f"\nFound {len(impossible_values)} physically impossible values")
if len(impossible_values) > 0:
    print("Constraint violations by variable:")
    for var, count in constraint_violations.items():
        print(f"  {var}: {count} violations")
    # Show sample of impossible values
    print("\nSample impossible values:")
    sample_cols = ['City', 'DateTime', 'constraint_violation']
    print(impossible_values[sample_cols].head(5))

# 4. VISUALIZATION (save to files)
print("\n==== GENERATING VISUALIZATIONS ====")

# Plot demand time series with anomalies highlighted for a sample city
def plot_demand_anomalies(df, city, anomalies, save_path):
    plt.figure(figsize=(15, 6))
    
    city_data = df[df['City'] == city]
    city_anomalies = anomalies[anomalies['City'] == city]
    
    plt.plot(city_data['DateTime'], city_data['Demand (MW)'], 'b-', alpha=0.7)
    if len(city_anomalies) > 0:
        plt.scatter(city_anomalies['DateTime'], city_anomalies['Demand (MW)'], 
                    color='red', s=50, label='Anomalies')
    
    plt.title(f'Electricity Demand for {city} with Anomalies Highlighted')
    plt.xlabel('Date')
    plt.ylabel('Demand (MW)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"Saved demand anomaly plot for {city} to {save_path}")

# Boxplots for weather variables by city
def plot_weather_boxplots(df, save_path):
    weather_vars = ['Temperature (F)', 'Humidity', 'Wind Speed (mph)', 'Pressure']
    for var in weather_vars:
        if var not in df.columns:
            continue
            
        plt.figure(figsize=(14, 8))
        sns.boxplot(x='City', y=var, data=df)
        plt.title(f'Distribution of {var} by City')
        plt.xticks(rotation=45)
        plt.tight_layout()
        var_name = var.replace(" ", "_").replace("(", "").replace(")", "")
        file_path = f"{save_path}_{var_name}.png"
        plt.savefig(file_path)
        plt.close()
        print(f"Saved boxplot for {var} to {file_path}")

# Create visualizations for a few sample cities
if len(df['City'].unique()) > 0:
    for i, city in enumerate(df['City'].unique()[:3]):  # First 3 cities
        plot_demand_anomalies(df, city, demand_anomalies, f"demand_anomalies_{city}.png")
    
    plot_weather_boxplots(df, "weather_boxplot")

print("\nAnalysis complete! Check the output files for visualizations.")

Loading dataset...

Dataset shape: (165192, 22)
First few rows:
            Local Time Balancing Authority   Data Date  Hour Number  \
0  2018-07-01 06:00:00                CISO  2018-07-01          6.0   
1  2018-07-01 07:00:00                CISO  2018-07-01          7.0   

  Sub-Region  Demand (MW) City  Temperature (F)  Humidity  Wind Speed (mph)  \
0        SCE       9472.0   la            65.45      0.79              4.23   
1        SCE       9353.0   la            65.45      0.79              4.23   

   ... UV Index  Visibility  Cloud Cover  Hour  Day  Month  Weekday  Weekend  \
0  ...      0.0       9.798         0.25     6    1      7        6        1   
1  ...      0.0       9.798         0.25     7    1      7        6        1   

   Season             DateTime  
0       3  2018-07-01 06:00:00  
1       3  2018-07-01 07:00:00  

[2 rows x 22 columns]

Converting date and time columns...


C:\Users\leads trading\AppData\Local\Temp\ipykernel_3356\3761659935.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['Data Date'] + ' ' + df['Hour Number'].astype(str) + ':00', errors='coerce')


DateTime conversion successful: True
Sample DateTime values: [Timestamp('2018-07-01 06:00:00'), Timestamp('2018-07-01 07:00:00'), Timestamp('2018-07-01 08:00:00')]

==== CHECKING TIME INTERVAL CONSISTENCY ====
Time interval consistency by city:

la:
  Consistent intervals: False
  Expected interval: 0 days 01:00:00
  Found 1376 irregular intervals
  Sample irregular intervals: [Timedelta('0 days 02:00:00'), Timedelta('0 days 00:00:00'), Timedelta('0 days 02:00:00'), Timedelta('0 days 00:00:00'), Timedelta('0 days 02:00:00')]

san_diego:
  Consistent intervals: False
  Expected interval: 0 days 01:00:00
  Found 1376 irregular intervals
  Sample irregular intervals: [Timedelta('0 days 02:00:00'), Timedelta('0 days 00:00:00'), Timedelta('0 days 02:00:00'), Timedelta('0 days 00:00:00'), Timedelta('0 days 02:00:00')]

san_jose:
  Consistent intervals: False
  Expected interval: 0 days 01:00:00
  Found 1376 irregular intervals
  Sample irregular intervals: [Timedelta('0 days 02:00:00'), Time

In [5]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import timedelta

def load_and_preprocess(filepath):
    """Load and preprocess the dataset."""
    df = pd.read_csv(filepath)
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df['Local Time'] = pd.to_datetime(df['Local Time'])
    return df

def fix_time_intervals(df):
    """Resample data to ensure consistent hourly intervals."""
    cleaned_dfs = []
    
    for city in df['City'].unique():
        city_df = df[df['City'] == city].copy()
        city_df = city_df.set_index('DateTime')
        
        # Create complete hourly index
        full_range = pd.date_range(
            start=city_df.index.min(),
            end=city_df.index.max(),
            freq='H'
        )
        
        # Resample and interpolate
        city_df = city_df.reindex(full_range)
        
        # Interpolate numerical columns
        num_cols = ['Demand (MW)', 'Temperature (F)', 'Humidity', 'Wind Speed (mph)',
                   'Pressure', 'UV Index', 'Visibility', 'Cloud Cover']
        city_df[num_cols] = city_df[num_cols].interpolate(method='time')
        
        # Forward fill categorical columns
        cat_cols = ['Balancing Authority', 'Sub-Region', 'Weather Summary']
        city_df[cat_cols] = city_df[cat_cols].ffill()
        
        # Backfill remaining missing values
        city_df = city_df.bfill()
        
        # Reset city info
        city_df['City'] = city
        city_df['Data Date'] = city_df.index.date
        city_df['Hour'] = city_df.index.hour
        city_df['Day'] = city_df.index.day
        city_df['Month'] = city_df.index.month
        city_df['Weekday'] = city_df.index.weekday
        city_df['Weekend'] = city_df.index.weekday.isin([5,6]).astype(int)
        city_df['Season'] = (city_df.index.month % 12 + 3) // 3
        
        cleaned_dfs.append(city_df.reset_index().rename(columns={'index':'DateTime'}))
    
    return pd.concat(cleaned_dfs, ignore_index=True)

def correct_demand_anomalies(df):
    """Correct demand anomalies using city-specific thresholds."""
    df = df.copy()
    demand_log = []
    
    for city in df['City'].unique():
        city_mask = df['City'] == city
        temp_df = df[city_mask].copy()
        temp_df = temp_df.set_index('DateTime')
        
        # Calculate robust statistics
        q1, q3 = np.percentile(temp_df['Demand (MW)'], [25, 75])
        iqr = q3 - q1
        lower_bound = q1 - 3 * iqr
        upper_bound = q3 + 3 * iqr
        
        # Find anomalies
        anomalies = (temp_df['Demand (MW)'] < lower_bound) | (temp_df['Demand (MW)'] > upper_bound)
        anomaly_count = anomalies.sum()
        
        if anomaly_count > 0:
            for dt in temp_df.index[anomalies]:
                hour = dt.hour
                date_range = (dt - timedelta(days=3), dt + timedelta(days=3))
                
                # Get same hour from ±3 days
                window_mask = (
                    (temp_df.index.hour == hour) & 
                    (temp_df.index >= date_range[0]) & 
                    (temp_df.index <= date_range[1])
                )
                window = temp_df[window_mask]
                
                if len(window) > 0:
                    median_val = window['Demand (MW)'].median()
                    temp_df.loc[dt, 'Demand (MW)'] = median_val
            
            # Update original DataFrame
            df.loc[city_mask, 'Demand (MW)'] = temp_df['Demand (MW)'].values
            demand_log.append(f"Corrected {anomaly_count} demand anomalies in {city}")
    
    return df, demand_log

def correct_weather_anomalies(df):
    """Correct weather anomalies using physical constraints."""
    df = df.copy()
    weather_log = []
    
    # City-specific temperature bounds (Fahrenheit)
    temp_bounds = {
        'la': (40, 110),
        'san_diego': (40, 105),
        'san_jose': (35, 105),
        'dallas': (10, 110),
        'houston': (20, 105),
        'san_antonio': (25, 108),
        'nyc': (10, 100),
        'philadelphia': (10, 100),
        'phoenix': (40, 120),
        'seattle': (30, 95)
    }
    
    for city in df['City'].unique():
        city_mask = df['City'] == city
        temp_df = df[city_mask].copy()
        temp_df = temp_df.set_index('DateTime')
        
        # Fix temperature anomalies
        lower, upper = temp_bounds[city]
        temp_anomalies = (temp_df['Temperature (F)'] < lower) | (temp_df['Temperature (F)'] > upper)
        temp_count = temp_anomalies.sum()
        
        if temp_count > 0:
            temp_df.loc[temp_anomalies, 'Temperature (F)'] = np.nan
            temp_df['Temperature (F)'] = temp_df['Temperature (F)'].interpolate(method='time')
            weather_log.append(f"Corrected {temp_count} temperature anomalies in {city}")
        
        # Fix other weather variables
        weather_vars = ['Humidity', 'Wind Speed (mph)', 'Pressure']
        for var in weather_vars:
            # Calculate IQR bounds
            q1, q3 = np.percentile(temp_df[var].dropna(), [25, 75])
            iqr = q3 - q1
            lower_bound = q1 - 3 * iqr
            upper_bound = q3 + 3 * iqr
            
            # Find anomalies
            anomalies = (temp_df[var] < lower_bound) | (temp_df[var] > upper_bound)
            anomaly_count = anomalies.sum()
            
            if anomaly_count > 0:
                temp_df.loc[anomalies, var] = np.nan
                temp_df[var] = temp_df[var].interpolate(method='time')
                weather_log.append(f"Corrected {anomaly_count} {var} anomalies in {city}")
        
        # Update original DataFrame
        for col in ['Temperature (F)'] + weather_vars:
            df.loc[city_mask, col] = temp_df[col].values
    
    return df, weather_log

def save_cleaned_data(df, filepath):
    """Save cleaned data to CSV."""
    # Reorder columns to match original structure
    original_order = [
        'Local Time', 'Balancing Authority', 'Data Date', 'Hour Number', 'Sub-Region',
        'Demand (MW)', 'City', 'Temperature (F)', 'Humidity', 'Wind Speed (mph)',
        'Weather Summary', 'Pressure', 'UV Index', 'Visibility', 'Cloud Cover',
        'Hour', 'Day', 'Month', 'Weekday', 'Weekend', 'Season', 'DateTime'
    ]
    df = df[original_order]
    df.to_csv(filepath, index=False)

def main():
    # Load data
    print("Loading and preprocessing data...")
    df = load_and_preprocess("merged_weather_demand_final.csv")
    
    # Fix time intervals
    print("\nFixing time intervals...")
    df = fix_time_intervals(df)
    
    # Correct demand anomalies
    print("\nCorrecting demand anomalies...")
    df, demand_log = correct_demand_anomalies(df)
    
    # Correct weather anomalies
    print("\nCorrecting weather anomalies...")
    df, weather_log = correct_weather_anomalies(df)
    
    # Save cleaned data
    print("\nSaving cleaned data...")
    save_cleaned_data(df, "merged_weather_demand_final.csv")
    
    # Print summary
    print("\n=== Cleaning Summary ===")
    print("\nDemand Anomalies Corrected:")
    for log in demand_log:
        print(f"- {log}")
    
    print("\nWeather Anomalies Corrected:")
    for log in weather_log:
        print(f"- {log}")
    
    print(f"\nFinal dataset shape: {df.shape}")
    print("\nData cleaning complete. Saved to merged_weather_demand_final.csv")

if __name__ == "__main__":
    main()

Loading and preprocessing data...

Fixing time intervals...

Correcting demand anomalies...

Correcting weather anomalies...

Saving cleaned data...

=== Cleaning Summary ===

Demand Anomalies Corrected:
- Corrected 1729 demand anomalies in seattle

Weather Anomalies Corrected:

Final dataset shape: (165192, 22)

Data cleaning complete. Saved to merged_weather_demand_final.csv


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

def verify_cleaned_data(filepath):
    """Run comprehensive verification checks on cleaned data"""
    print(" Starting data verification...")
    df = pd.read_csv(filepath)
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df['Local Time'] = pd.to_datetime(df['Local Time'])
    
    verification_results = {
        'timestamp_checks': verify_timestamps(df),
        'value_ranges': verify_value_ranges(df),
        'missing_data': verify_missing_data(df),
        'anomaly_check': verify_anomalies(df),
        'city_consistency': verify_city_consistency(df)
    }
    
    print("\n=== VERIFICATION SUMMARY ===")
    for check_name, results in verification_results.items():
        print(f"\n{check_name.upper()} RESULTS:")
        if isinstance(results, dict):
            for k, v in results.items():
                print(f"- {k}: {v}")
        else:
            print(results)
    
    return verification_results

def verify_timestamps(df):
    """Verify timestamp continuity and consistency"""
    results = {}
    
    # Check for missing hours
    for city in df['City'].unique():
        city_df = df[df['City'] == city].set_index('DateTime')
        expected_hours = pd.date_range(
            start=city_df.index.min(),
            end=city_df.index.max(),
            freq='H'
        )
        missing = expected_hours.difference(city_df.index)
        results[f"{city}_missing_hours"] = len(missing)
    
    # Check for duplicates
    duplicates = df.duplicated(subset=['City', 'DateTime'], keep=False).sum()
    results['duplicate_records'] = duplicates
    
    # Check time intervals
    interval_issues = 0
    for city in df['City'].unique():
        city_df = df[df['City'] == city].sort_values('DateTime')
        time_diffs = city_df['DateTime'].diff().dropna()
        irregular = time_diffs[time_diffs != timedelta(hours=1)]
        interval_issues += len(irregular)
    results['irregular_intervals'] = interval_issues
    
    return results

def verify_value_ranges(df):
    """Verify all values are within reasonable ranges"""
    results = {}
    
    # Demand checks
    demand_stats = df['Demand (MW)'].agg(['min', 'max', 'mean'])
    results['demand_stats'] = demand_stats.to_dict()
    
    # Temperature checks
    temp_stats = df.groupby('City')['Temperature (F)'].agg(['min', 'max'])
    results['temperature_ranges'] = temp_stats.to_dict('index')
    
    # Humidity checks (0-100%)
    bad_humidity = df[(df['Humidity'] < 0) | (df['Humidity'] > 1)].shape[0]
    results['invalid_humidity'] = bad_humidity
    
    return results

def verify_missing_data(df):
    """Check for remaining missing values"""
    results = {}
    
    numeric_cols = ['Demand (MW)', 'Temperature (F)', 'Humidity', 
                   'Wind Speed (mph)', 'Pressure', 'UV Index']
    
    for col in numeric_cols:
        missing = df[col].isna().sum()
        results[f"missing_{col}"] = missing
    
    return results

def verify_anomalies(df, z_threshold=3):
    """Check for remaining statistical anomalies"""
    results = {}
    
    # Demand anomalies
    for city in df['City'].unique():
        city_demand = df[df['City'] == city]['Demand (MW)']
        z_scores = np.abs((city_demand - city_demand.mean()) / city_demand.std())
        anomalies = z_scores > z_threshold
        results[f"{city}_demand_anomalies"] = anomalies.sum()
    
    # Weather anomalies
    weather_vars = ['Temperature (F)', 'Humidity', 'Wind Speed (mph)']
    for var in weather_vars:
        anomalies = 0
        for city in df['City'].unique():
            city_data = df[df['City'] == city][var]
            q1, q3 = np.percentile(city_data, [25, 75])
            iqr = q3 - q1
            lower = q1 - 3*iqr
            upper = q3 + 3*iqr
            anomalies += ((city_data < lower) | (city_data > upper)).sum()
        results[f"{var}_anomalies"] = anomalies
    
    return results

def verify_city_consistency(df):
    """Verify all cities have consistent data coverage"""
    results = {}
    
    city_stats = df.groupby('City')['DateTime'].agg(['min', 'max', 'count'])
    results['date_ranges'] = city_stats.to_dict('index')
    
    # Check if all cities have same date range
    start_dates = city_stats['min'].unique()
    end_dates = city_stats['max'].unique()
    results['consistent_date_ranges'] = len(start_dates) == 1 and len(end_dates) == 1
    
    return results

if __name__ == "__main__":
    verification_results = verify_cleaned_data("merged_weather_demand_final.csv")

🔍 Starting data verification...

=== VERIFICATION SUMMARY ===

TIMESTAMP_CHECKS RESULTS:
- la_missing_hours: 0
- san_diego_missing_hours: 0
- san_jose_missing_hours: 0
- dallas_missing_hours: 0
- houston_missing_hours: 0
- san_antonio_missing_hours: 0
- nyc_missing_hours: 0
- philadelphia_missing_hours: 0
- phoenix_missing_hours: 0
- seattle_missing_hours: 0
- duplicate_records: 0
- irregular_intervals: 0

VALUE_RANGES RESULTS:
- demand_stats: {'min': 58.0, 'max': 20017.0, 'mean': 4163.547514407477}
- temperature_ranges: {'dallas': {'min': 22.24, 'max': 109.31}, 'houston': {'min': 30.15, 'max': 100.84}, 'la': {'min': 40.49, 'max': 98.92}, 'nyc': {'min': 10.46, 'max': 96.25}, 'philadelphia': {'min': 10.24, 'max': 97.93}, 'phoenix': {'min': 40.0, 'max': 114.03}, 'san_antonio': {'min': 25.51, 'max': 106.11}, 'san_diego': {'min': 40.12, 'max': 90.63}, 'san_jose': {'min': 35.06, 'max': 93.41}, 'seattle': {'min': 30.02, 'max': 90.32}}
- invalid_humidity: 0

MISSING_DATA RESULTS:
- missing_De

In [2]:
import pandas as pd

def remove_column_and_save(filepath, column_to_remove='Local Time'):
    """
    Remove specified column from CSV and save back to same file.
    
    Args:
        filepath (str): Path to the CSV file
        column_to_remove (str): Name of column to remove (default: 'Local Time')
    """
    # Load the data
    df = pd.read_csv(filepath)
    
    # Check if column exists
    if column_to_remove not in df.columns:
        print(f"Column '{column_to_remove}' not found in the file.")
        print("Available columns:", list(df.columns))
        return
    
    # Remove the column
    df = df.drop(columns=[column_to_remove])
    
    # Save back to same CSV file
    df.to_csv(filepath, index=False)
    print(f"Column '{column_to_remove}' removed and file saved successfully.")

if __name__ == "__main__":
    # Replace with your actual file path
    csv_file = "merged_weather_demand_final.csv"
    remove_column_and_save(csv_file)

Column 'Local Time' removed and file saved successfully.


In [2]:
import pandas as pd
import numpy as np

def print_min_max_values(filepath):
    # Load the data
    df = pd.read_csv(filepath)
    
    print("Minimum and Maximum values for each column:\n")
    print("{:<25} {:<15} {:<15}".format('Column', 'Min', 'Max'))
    print("-" * 55)
    
    for column in df.columns:
        try:
            # For numeric columns
            if pd.api.types.is_numeric_dtype(df[column]):
                min_val = df[column].min()
                max_val = df[column].max()
                print("{:<25} {:<15} {:<15}".format(column, min_val, max_val))
            
            # For datetime columns
            elif pd.api.types.is_datetime64_any_dtype(df[column]):
                min_val = df[column].min()
                max_val = df[column].max()
                print("{:<25} {:<15} {:<15}".format(column, min_val, max_val))
            
            # For string/categorical columns
            else:
                unique_count = df[column].nunique()
                if unique_count <= 10:  # Show values if few unique
                    min_val = df[column].min()
                    max_val = df[column].max()
                    print("{:<25} {:<15} {:<15}".format(column, f"'{min_val}'", f"'{max_val}'"))
                else:
                    print("{:<25} {:<15} {:<15}".format(column, f"{unique_count} unique", "values"))
        
        except Exception as e:
            print("{:<25} {}".format(column, f"Error: {str(e)}"))

if __name__ == "__main__":
    # Replace with your actual file path
    csv_file = "merged_weather_demand_final.csv"
    print_min_max_values(csv_file)

Minimum and Maximum values for each column:

Column                    Min             Max            
-------------------------------------------------------
Balancing Authority       'CISO'          'PNM'          
Date                      689 unique      values         
Hour Number               1               23             
Sub-Region                'NRTH'          'ZONA'         
Demand (MW)               58.0            20017.0        
City                      'dallas'        'seattle'      
Temperature (F)           10.24           114.03         
Humidity                  0.0             1.0            
Wind Speed (mph)          0.0             21.9           
Weather Summary           39 unique       values         
Pressure                  990.7           1042.1         
UV Index                  0.0             12.0           
Visibility                0.215           10.0           
Cloud Cover               0.0             1.0            
Hour                      0  

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_weather_demand_final.csv')

# Function to replace outliers with column mean
def replace_outliers_with_mean(df, column):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # Calculate the IQR (Interquartile Range)
    IQR = Q3 - Q1
    
    # Define the bounds for non-outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Replace outliers with the mean of the column
    df[column] = df[column].apply(lambda x: df[column].mean() if x < lower_bound or x > upper_bound else x)
    
    return df

# Replace outliers in 'Demand (MW)' and 'Temperature (F)' columns
df = replace_outliers_with_mean(df, 'Demand (MW)')
df = replace_outliers_with_mean(df, 'Temperature (F)')

# Save the modified DataFrame back to the original CSV file
df.to_csv('merged_weather_demand_final.csv', index=False)

print("Outliers have been replaced with the mean. The updated file has been saved back to 'merged_weather_demand_final.csv'.")


Outliers have been replaced with the mean. The updated file has been saved back to 'merged_weather_demand_final.csv'.


In [3]:
#AGGREGATION
import pandas as pd

# Load your dataset
df = pd.read_csv('merged_weather_demand_final.csv')

# Ensure proper datetime conversion
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Weekly aggregation
weekly_stats = df.groupby(['City', 'Balancing Authority', 'Sub-Region', 
                          pd.Grouper(key='DateTime', freq='W-MON')]).agg({
    'Demand (MW)': ['mean', 'max', 'min', 'sum', 'std', 'median'],
    'Temperature (F)': ['mean', 'max', 'min'],
    'Humidity': 'mean',
    'Wind Speed (mph)': 'mean',
    'Weekend': 'sum'
}).reset_index()

# FLATTEN COLUMN NAMES PROPERLY
weekly_stats.columns = ['_'.join(col).strip() for col in weekly_stats.columns.values]
weekly_stats = weekly_stats.rename(columns={
    'DateTime_': 'Week_Start_Date',
    'Demand (MW)_mean': 'Avg_Demand_MW',
    'Demand (MW)_max': 'Peak_Demand_MW',
    'Demand (MW)_min': 'Min_Demand_MW',
    'Demand (MW)_sum': 'Total_Weekly_Demand_MWh',
    'Demand (MW)_std': 'Demand_StDev',
    'Demand (MW)_median': 'Median_Demand_MW',
    'Temperature (F)_mean': 'Avg_Temp_F',
    'Temperature (F)_max': 'Max_Temp_F',
    'Temperature (F)_min': 'Min_Temp_F',
    'Humidity_mean': 'Avg_Humidity',
    'Wind Speed (mph)_mean': 'Avg_Wind_Speed_mph',
    'Weekend_sum': 'Weekend_Hours_Count'
})

# Now safely calculate derived metrics
weekly_stats['Demand_Variability'] = weekly_stats['Demand_StDev'] / weekly_stats['Avg_Demand_MW']
weekly_stats['Peak_to_Avg_Ratio'] = weekly_stats['Peak_Demand_MW'] / weekly_stats['Avg_Demand_MW']

# Save the results
#weekly_stats.to_csv('weekly_demand_stats_corrected.csv', index=False)

print("Successfully computed weekly statistics with columns:")
print(weekly_stats.columns.tolist())

Successfully computed weekly statistics with columns:
['City_', 'Balancing Authority_', 'Sub-Region_', 'Week_Start_Date', 'Avg_Demand_MW', 'Peak_Demand_MW', 'Min_Demand_MW', 'Total_Weekly_Demand_MWh', 'Demand_StDev', 'Median_Demand_MW', 'Avg_Temp_F', 'Max_Temp_F', 'Min_Temp_F', 'Avg_Humidity', 'Avg_Wind_Speed_mph', 'Weekend_Hours_Count', 'Demand_Variability', 'Peak_to_Avg_Ratio']
